In [153]:
def create_tables_sql():
    return """
    create table salary (
    id int,
    employee_id int,
    amount int,
    pay_date date
    );
    create table employee (
    employee_id int,
    department_id int
    );
    """

In [154]:
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
c.executescript(create_tables_sql())
conn.commit()

In [157]:
def insert_data_sql() -> str:
    return"""
    insert into salary values (1,1, 9000,"2017-03-31");
    insert into salary values (2,2, 6000,"2017-03-31");
    insert into salary values (3,3, 10000,"2017-03-31");
    insert into salary values (4,1, 7000,"2017-02-28");
    insert into salary values (5,2, 6000,"2017-02-28");
    insert into salary values (6,3, 8000,"2017-02-28");
    insert into employee values (1,1);
    insert into employee values(2,2);
    insert into employee values (3,2);
    """

In [158]:
c.executescript(insert_data_sql())
conn.commit()

In [145]:
def get_sql():
    return """
        select distinct(s1.id), s1.visit_date,s1.people
        from stadium s1,
           stadium s2,
           stadium s3
        where 
            s1.people >= 100 and s2.people >= 100 and s3.people >= 100
            and (
            (s1.id = s2.id + 2 and s1.id = s3.id + 1) 
            or
            (s1.id = s2.id + 1 and s1.id = s3.id - 1)
            or
            (s1.id = s2.id - 1 and s1.id = s3.id -2)
            )
            order by s1.id;
    """

In [299]:
def get_sql():
    return """
        with average as (
            select s.pay_date,e1.department_id,
                avg(s.amount) as avg_salary
            from salary s
                join employee e1 on s.employee_id = e1.employee_id
                group by e1.department_id,s.pay_date
        ), total_avg as (
             select s.pay_date,avg(s.amount) as 'total'
             from salary s
             group by s.pay_date
        )
         select strftime('%Y-%m',s.pay_date) as pay_month,
            average.department_id,
            (case when average.avg_salary > total then 'higher'
                     when average.avg_salary < total then 'lower'
                     else 'same'
            end) as 'comparison'
         from salary s
             join average on s.pay_date = average.pay_date
             join total_avg on s.pay_date = total_avg.pay_date
             group by average.department_id,s.pay_date
             order by s.pay_date desc;
    """

In [300]:
c.execute(get_sql())
result = c.fetchall()
result

[('2017-03', 1, 'higher'),
 ('2017-03', 2, 'lower'),
 ('2017-02', 1, 'same'),
 ('2017-02', 2, 'same')]